

1. Pull all current sections enrollments that should be graded
2. Join with Platform Letter Grades
3. Convert letter grades to P/NP
4. Export CSV for SL review
5. Export TXT for student_grades import

In [23]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter

from sps_automation import legacy as sps

conf = sps.load_config('./config/config.ini')

In [24]:
# create a database connection object for CA database
#db_ca = sps.connect_to_ca_illuminate_db()
db_wa = sps.connect_to_wa_illuminate_db()

In [25]:
# query the Illuminate database to pull all "gradable" enrollments.
query = """
SELECT
  local_student_id || ' ' ||
    CASE WHEN right(school_course_id,1) = 'M' THEN left(school_course_id,length(school_course_id)-1) ELSE school_course_id END AS stu_crs_lookup
  , sites.site_name
  , students.local_student_id
  , students.last_name
  , students.first_name
  , (sterma.grade_level_id - 1) as current_grade_level
  , ssa.section_id
  , sections.local_section_id
  , departments.department_name
  , courses.school_course_id
  , courses.short_name
  , CASE WHEN sterma.grade_level_id < 10 AND courses.school_course_id LIKE 'I%' THEN 0.5 ELSE courses.variable_credits_high END AS max_credits
  , users.last_name || ', ' || users.first_name AS teacher
  , sites.site_id

FROM
  matviews.student_term_aff AS sterma

LEFT JOIN (SELECT * FROM terms WHERE terms.start_date >= '2018-08-15' AND terms.end_date <= '2019-06-22' AND terms.term_name = 'Year') AS sy19terms on sterma.term_id = sy19terms.term_id
LEFT JOIN (SELECT * FROM student_session_aff WHERE (student_session_aff.leave_date IS NULL OR student_session_aff.leave_date BETWEEN '2019-05-15' AND '2019-06-23')) AS ssessa ON ssessa.student_id = sterma.student_id
LEFT JOIN (SELECT * FROM sessions WHERE sessions.academic_year = 2019 AND sessions.session_type_id = 1) AS sy19sess ON ssessa.session_id = sy19sess.session_id
LEFT JOIN sites on sy19sess.site_id = sites.site_id
LEFT JOIN students on sterma.student_id = students.student_id
LEFT JOIN (SELECT * FROM section_student_aff WHERE entry_date >= '2018-08-15' AND (leave_date IS NULL or leave_date > '2019-05-15')) AS ssa on ssa.student_id  = sterma.student_id
LEFT JOIN sections on ssa.section_id = sections.section_id
LEFT JOIN courses on ssa.course_id = courses.course_id
LEFT JOIN departments on courses.department_id = departments.department_id
LEFT JOIN section_teacher_aff sta on sta.section_id = ssa.section_id
LEFT JOIN users on sta.user_id = users.user_id

WHERE
  courses.transcript_inclusion IS NOT FALSE
  AND courses.is_active IS TRUE
  AND courses.variable_credits_high  >= 0.5
  AND (sterma.leave_date > '2019-05-20' OR sterma.leave_date IS NULL)
  AND sterma.entry_date BETWEEN '2018-08-01' AND '2019-05-20'
  AND sta.primary_teacher IS TRUE
  AND (sta.end_date > '2019-06-01' OR sta.end_date IS NULL)
  
  
  AND sy19sess.site_id < 100
  AND sterma.grade_level_id = 13

--   AND students.local_student_id = '20916'

GROUP BY
  sites.site_name
  , students.local_student_id
  , students.last_name
  , students.first_name
  , sterma.grade_level_id
  , ssa.section_id
  , sections.local_section_id
  , departments.department_name
  , courses.school_course_id
  , courses.short_name
  , ssa.entry_date
  , ssa.leave_date
  , courses.variable_credits_high
  , users.last_name
  , users.first_name
  , sites.site_id

ORDER BY
  site_name
  , sterma.grade_level_id
  , last_name
  , first_name
  , school_course_id
"""

In [26]:
#Load query results as "gradable sections"
#gs_ca = pd.read_sql(query,db_ca['connection'])
gs_wa = pd.read_sql(query,db_wa['connection'])


#make sure 'local_student_id' is INT
#gs_ca['local_student_id'] = pd.to_numeric(gs_ca['local_student_id'])
gs_wa['local_student_id'] = pd.to_numeric(gs_wa['local_student_id'])

#gs_ca.shape
gs_wa.shape

(1018, 14)

In [27]:
#gs_all = pd.concat ([gs_ca,gs_wa])
gs_nodups = gs_wa.drop_duplicates(subset = 'stu_crs_lookup', keep = 'last')


gs_nodups.head(10)

#export csv
gs_nodups.to_csv('../data/SY19_midyear/Outputs/wa_gradable_sections_mid.csv')

gs_nodups.shape

(1010, 14)

In [28]:
#TODO: Read in the platform grades CSV to a dataframe called pl_grades and 

pl_dump = pd.read_csv('../data/SY19_midyear/Inputs/wa_platformgrades.csv', encoding='latin-1')

#drop duplicate records
pl_nodups = pl_dump.drop_duplicates(subset = 'stu_crs_lookup', keep = 'last')

#create a "is_failing" flag
fail_grades = ['I','F','NP']

def is_failing(corrected_grade):
    if corrected_grade in fail_grades:
        return True
    else: 
        return False

    
pl_nodups['is_failing?'] = pl_nodups['corrected_grade'].apply(is_failing)

#merge queried sections with platform courses

merged_grades = pd.merge(gs_nodups[['stu_crs_lookup','site_name','local_student_id','last_name','first_name','current_grade_level','section_id','local_section_id','department_name','school_course_id','short_name','max_credits','teacher','site_id']], 
                         pl_nodups[['stu_crs_lookup','grade_type','LETTER_GRADE','corrected_grade','is_failing?','inc_reason','date_of_pull']], on = 'stu_crs_lookup', how = 'left', suffixes=('_ill','_pltfrm'))

merged_grades.shape



C:\Users\Bobby Cupp\Anaconda3\envs\summit\lib\site-packages\ipykernel\__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(1010, 20)

In [29]:
#export csv
merged_grades.to_csv('../data/SY19_midyear/Outputs/wa_merged_grades_export_mid.csv')

#assert gs_nodups.shape[1] == merged_grades.shape[1]

In [30]:
# Create a simplified dataframe for viewing 

simple_df = merged_grades[['site_name','local_student_id','last_name','first_name','current_grade_level','department_name','school_course_id','short_name','teacher','date_of_pull']]

simple_pltfm = simple_df
simple_pltfm['LETTER_GRADE'] = merged_grades.LETTER_GRADE
simple_pltfm['corrected_grade'] = merged_grades.corrected_grade
simple_pltfm['Inc Reason'] = merged_grades.inc_reason
simple_pltfm['stu_crs_lookup'] = merged_grades.stu_crs_lookup

#export csv
simple_pltfm.to_csv('../data/SY19_midyear/Outputs/wa_simple_pltfm.csv')

simple_pltfm.head(10)

C:\Users\Bobby Cupp\Anaconda3\envs\summit\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Bobby Cupp\Anaconda3\envs\summit\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Bobby Cupp\Anaconda3\envs\summit\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

,site_name,local_student_id,last_name,first_name,current_grade_level,department_name,school_course_id,short_name,teacher,date_of_pull,LETTER_GRADE,corrected_grade,Inc Reason,stu_crs_lookup
0,Summit Public School: Olympus,120029,Aguilera-Ocampo,C. Mateo,12,History,A400,AP Government,"Crain, Carrie",1/15/2019 12:00:00 AM,B-,B-,PFA?[_] Proj Score?[_] Ovrdue Proj?[_],120029 A400
1,Summit Public School: Olympus,120029,Aguilera-Ocampo,C. Mateo,12,English,B400,AP Literature and Composition,"Leon, Jacob",1/15/2019 12:00:00 AM,A-,A-,PFA?[_] Proj Score?[_] Ovrdue Proj?[_],120029 B400
2,Summit Public School: Olympus,120029,Aguilera-Ocampo,C. Mateo,12,Science,D430,AP Environmental Science,"Prohuska, Jordan",1/15/2019 12:00:00 AM,B+,B+,PFA?[_] Proj Score?[_] Ovrdue Proj?[_],120029 D430
3,Summit Public School: Olympus,120029,Aguilera-Ocampo,C. Mateo,12,Electives,F502,Intro to Visual Art,"Guerra, Kristen",1/15/2019 12:00:00 AM,A,A,PFA?[_] Proj Score?[_] Ovrdue Proj?[_],120029 F502
4,Summit Public School: Olympus,120029,Aguilera-Ocampo,C. Mateo,12,Electives,I1043,Conditioning,"Ewing, Jasmine",1/15/2019 12:00:00 AM,A+,P,PFA?[_] Proj Score?[_] Ovrdue Proj?[_],120029 I1043
5,Summit Public School: Olympus,120029,Aguilera-Ocampo,C. Mateo,12,Intervention,S4100,Bridge to College Math,"Harmon, Sydney",1/15/2019 12:00:00 AM,A+,A+,PFA?[_] Proj Score?[_] Ovrdue Proj?[_],120029 S4100
6,Summit Public School: Olympus,120376,Ahmed,Salahuddin,12,History,A400,AP Government,"Crain, Carrie",1/15/2019 12:00:00 AM,B+,B+,PFA?[_] Proj Score?[_] Ovrdue Proj?[_],120376 A400
7,Summit Public School: Olympus,120376,Ahmed,Salahuddin,12,English,B400,AP Literature and Composition,"Leon, Jacob",1/15/2019 12:00:00 AM,B,B,PFA?[_] Proj Score?[_] Ovrdue Proj?[_],120376 B400
8,Summit Public School: Olympus,120376,Ahmed,Salahuddin,12,Math,C402,Modeling & Stats,"Hansen, Alison",1/15/2019 12:00:00 AM,A-,A-,PFA?[_] Proj Score?[_] Ovrdue Proj?[_],120376 C402
9,Summit Public School: Olympus,120376,Ahmed,Salahuddin,12,Science,D430,AP Environmental Science,"Prohuska, Jordan",1/15/2019 12:00:00 AM,B-,B-,PFA?[_] Proj Score?[_] Ovrdue Proj?[_],120376 D430


In [31]:
# Generate a list of stu/sections without a platform grade

ungraded_sects = simple_df[pd.isnull(merged_grades.LETTER_GRADE)]

ungraded_sects.sort_values(['teacher','site_name','school_course_id'], ascending=[True,True,True])

#export csv
ungraded_sects.to_csv('../data/SY19_midyear/Outputs/wa_ungraded_sections.csv')

ungraded_sects.shape

(205, 14)

In [32]:
# Generate a list of SENIOR stu/sections with I/F Grades

senior_fails = simple_pltfm[(simple_pltfm.LETTER_GRADE == 'I') & (simple_pltfm.current_grade_level == 12) & (simple_df.local_student_id != 40477
)]

senior_fails.sort_values(['site_name','local_student_id'], ascending=[True,True])


#export csv\
senior_fails.to_csv('../data/SY19_midyear/Outputs/wa_senior_fails.csv')

senior_fails.shape

(134, 14)

In [33]:
# Generate a student_grades.txt file for import to Illuminate

student_grades_pieces = merged_grades[['local_student_id','site_id','local_section_id','school_course_id', 'current_grade_level','corrected_grade','max_credits','is_failing?']]

student_grades_pieces['grading_period_name'] = 'Final Grades'
student_grades_pieces['academic_year'] = '2019'
student_grades_pieces['credits_received'] = np.where(student_grades_pieces['is_failing?'] == True, 0, student_grades_pieces.max_credits)
student_grades_pieces['is_final?'] = 0
student_grades_pieces['is_repeat?'] = 0
student_grades_pieces['session_type_id'] = 1

student_grades_txt = student_grades_pieces[['local_student_id','site_id','grading_period_name','academic_year','local_section_id','school_course_id','corrected_grade','max_credits','credits_received','is_final?','is_repeat?','session_type_id']]
student_grades_txt.head(10)

#export csv
student_grades_txt.to_csv('../data/SY19_midyear/Outputs/wa_student_grades.csv')

student_grades_txt.head(10)

C:\Users\Bobby Cupp\Anaconda3\envs\summit\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Bobby Cupp\Anaconda3\envs\summit\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Bobby Cupp\Anaconda3\envs\summit\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

,local_student_id,site_id,grading_period_name,academic_year,local_section_id,school_course_id,corrected_grade,max_credits,credits_received,is_final?,is_repeat?,session_type_id
0,120029,12,Final Grades,2019,A400-Crain-Gold 2,A400,B-,1.0,1.0,0,0,1
1,120029,12,Final Grades,2019,B400-Leon-Gold 3,B400,A-,1.0,1.0,0,0,1
2,120029,12,Final Grades,2019,D430-Prohuska-Blue 3,D430,B+,1.0,1.0,0,0,1
3,120029,12,Final Grades,2019,F502-Guerra-Gold 4,F502,A,1.0,1.0,0,0,1
4,120029,12,Final Grades,2019,I1043-Breneman-Blue 4,I1043,P,1.0,1.0,0,0,1
5,120029,12,Final Grades,2019,S4100-Harmon-Blue 2,S4100,A+,1.0,1.0,0,0,1
6,120376,12,Final Grades,2019,A400-Crain-Gold 4,A400,B+,1.0,1.0,0,0,1
7,120376,12,Final Grades,2019,B400-Leon-Gold 2,B400,B,1.0,1.0,0,0,1
8,120376,12,Final Grades,2019,C402-Hansen-Blue 3,C402,A-,1.0,1.0,0,0,1
9,120376,12,Final Grades,2019,D430-Prohuska-Blue 4,D430,B-,1.0,1.0,0,0,1


# This part is for after you have imported all grades - generate lists for school review/auditing.

In [ ]:
# Generate a list of all Final Grades for this year.
query_final = """
SELECT
  local_student_id || ' ' ||
    CASE WHEN right(school_course_id,1) = 'M' THEN left(school_course_id,length(school_course_id)-1) ELSE school_course_id END AS stu_crs_lookup
  , sites.site_name
  , sites.site_id
  , students.local_student_id
  , students.last_name || ', ' || students.first_name AS name
  , (sterma.grade_level_id - 1) as current_grade_level
  , ssa.section_id
  , courses.school_course_id
  , courses.short_name
  , departments.department_name
  , users.last_name || ', ' || users.first_name AS teacher
  , grades.grade
  , sg.credits_received
  , sg.credits_possible

FROM
  matviews.student_term_aff AS sterma

LEFT JOIN (SELECT * FROM terms WHERE terms.start_date = '2017-08-15' AND terms.end_date = '2018-06-07' AND terms.term_name = 'Year') AS sy18terms on sterma.term_id = sy18terms.term_id
LEFT JOIN (SELECT * FROM student_session_aff WHERE student_session_aff.leave_date BETWEEN '2018-05-15' AND '2018-06-15') AS ssessa ON ssessa.student_id = sterma.student_id
LEFT JOIN (SELECT * FROM sessions WHERE sessions.academic_year = 2018 AND sessions.session_type_id = 1) AS sy18sess ON ssessa.session_id = sy18sess.session_id
LEFT JOIN sites on sy18sess.site_id = sites.site_id
LEFT JOIN students on sterma.student_id = students.student_id
LEFT JOIN (SELECT * FROM section_student_aff WHERE entry_date >= '2017-08-15' AND (leave_date IS NULL or leave_date > '2018-05-15')) AS ssa on ssa.student_id  = sterma.student_id
LEFT JOIN sections on ssa.section_id = sections.section_id
LEFT JOIN courses on ssa.course_id = courses.course_id
LEFT JOIN departments on courses.department_id = departments.department_id
LEFT JOIN section_teacher_aff sta on sta.section_id = ssa.section_id
LEFT JOIN users on sta.user_id = users.user_id
LEFT JOIN section_grading_period_aff sgpa on sections.section_id = sgpa.section_id
LEFT JOIN student_grades sg on sgpa.sgpa_id = sg.sgpa_id AND sg.student_id = students.student_id
LEFT JOIN grades on grades.grade_id = sg.grade_id

WHERE
  courses.transcript_inclusion IS NOT FALSE
  AND courses.is_active IS TRUE
  AND courses.variable_credits_high  >= 0.5
  AND sterma.leave_date > '2018-05-20' AND sterma.entry_date < '2018-05-20'
  AND sta.primary_teacher IS TRUE
  AND (sta.end_date > '2018-06-01' OR sta.end_date IS NULL)

  AND grades.grade IS NOT NULL
  AND sg.is_final IS TRUE

--   AND students.local_student_id = '20916'

GROUP BY
sites.site_name
  , sites.site_id
  , students.local_student_id
  , students.last_name
  , students.first_name
  , sterma.grade_level_id
  , ssa.section_id
  , courses.school_course_id
  , courses.short_name
  , departments.department_name
  , users.last_name
  , users.first_name
  , grades.grade
  , sg.credits_received
  , sg.credits_possible

ORDER BY
  sites.site_id
  , sterma.grade_level_id
  , current_grade_level
  , name
  , school_course_id
"""

In [ ]:
#Load query results as "posted_grades"
#pgraw_ca = pd.read_sql(query_final,db_ca['connection'])
pgraw_wa = pd.read_sql(query_final,db_wa['connection'])


#make sure 'local_student_id' is INT
#pgraw_ca['local_student_id'] = pd.to_numeric(pgraw_ca['local_student_id'])
pgraw_wa['local_student_id'] = pd.to_numeric(pg_wa['local_student_id'])

#remove duplicates
pg_wa = pgraw_wa.drop_duplicates(subset = 'stu_crs_lookup', keep = 'last')
pg_wa = pg_wa[pg_wa.columns[1:]]
pg_wa.head(5)
pg_wa.shape





In [ ]:
#export all to csv
pg_wa.to_csv('../data/SY19_midyear/Outputs/wa_postedgrades.csv')

#Sierra Grades
si_index = pg_wa['site_id'] == 11
pg_si = pg_wa[si_index]
inc_si_index = pg_si['grade'].isin(['F','I','NP'])
inc_si = pg_si[inc_si_index]

grades_by_gl_si = pg_si.pivot_table(index='current_grade_level',
                                      columns='grade',
                                      values='local_student_id',
                                      aggfunc='count',
                                      margins=True)
grades_by_gl_si = grades_by_gl_si.fillna(0).astype(int)

grades_by_course_si = pg_si.pivot_table(index=['department_name','short_name'],
                                      columns='grade',
                                      values='local_student_id',
                                      aggfunc='count',
                                      margins=True)
grades_by_course_si = grades_by_course_si.fillna(0).astype(int)

#export to Excel
writer = ExcelWriter('../data/SY19_midyear/Outputs/posted_grades/midyear_grades_sy19_sierra.xlsx')
pg_si.to_excel(writer,'all grades',index=False)
inc_si.to_excel(writer,'incompletes',index=False)
grades_by_gl_si.to_excel(writer,'grades by grade level')
grades_by_course_si.to_excel(writer,'grades by course')
writer.save()

In [ ]:
#Olympus Grades
ol_index = pg_wa['site_id'] == 12
pg_ol = pg_wa[ol_index]
inc_ol_index = pg_ol['grade'].isin(['F','I','NP'])
inc_ol = pg_ol[inc_ol_index]

grades_by_gl_ol = pg_ol.pivot_olble(index='current_grade_level',
                                      columns='grade',
                                      values='local_student_id',
                                      aggfunc='count',
                                      margins=True)
grades_by_gl_ol = grades_by_gl_ol.fillna(0).astype(int)

grades_by_course_ol = pg_ol.pivot_olble(index=['department_name','short_name'],
                                      columns='grade',
                                      values='local_student_id',
                                      aggfunc='count',
                                      margins=True)
grades_by_course_ol = grades_by_course_ol.fillna(0).astype(int)

#export to Excel
writer = ExcelWriter('../data/SY19_midyear/Outputs/posted_grades/midyear_grades_sy19_olympus.xlsx')
pg_ol.to_excel(writer,'all grades',index=False)
inc_ol.to_excel(writer,'incompletes',index=False)
grades_by_gl_ol.to_excel(writer,'grades by grade level')
grades_by_course_ol.to_excel(writer,'grades by course')
writer.save()